# Keras Core CRF minimal example and backend switching

This notebook shows how to use the backend-independent CRF ops and layer with Keras Core.
It attempts to switch backends (TensorFlow, JAX, PyTorch) if available.

In [ ]:
import keras
from keras import ops as K
import numpy as np
from keras_crf.core_kops import crf_log_likelihood, crf_decode
from keras_crf.layers_core import KerasCoreCRF
print('Keras backend:', keras.config.backend())

In [ ]:
# Try switching backends (will only work if installed)
for backend in ['tensorflow','jax','torch']:
    try:
        keras.config.set_backend(backend)
        print('Switched to backend:', backend)
        break
    except Exception as e:
        print('Could not set backend', backend, e)
print('Using backend:', keras.config.backend())

## Ops usage

In [ ]:
B,T,N = 2,4,3
rng = np.random.default_rng(0)
potentials = K.convert_to_tensor(rng.normal(size=(B,T,N)).astype('float32'))
tags = K.convert_to_tensor(rng.integers(0,N,size=(B,T),dtype=np.int32))
lens = K.convert_to_tensor(np.array([4,3],dtype=np.int32))
trans = K.convert_to_tensor(rng.normal(size=(N,N)).astype('float32'))
ll = crf_log_likelihood(potentials, tags, lens, trans)
dec, score = crf_decode(potentials, lens, trans)
print('log-likelihood:', K.convert_to_numpy(ll))
print('decoded:', K.convert_to_numpy(dec))
print('score:', K.convert_to_numpy(score))

## Layer usage

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(T,N), dtype='float32')
crf = KerasCoreCRF(units=N, use_kernel=False, use_boundary=False)
decoded, potentials, seq_len, trans_w = crf(inputs)
model = keras.Model(inputs, [decoded, potentials, seq_len, trans_w])
out = model.predict(np.random.randn(B,T,N).astype('float32'), verbose=0)
print([x.shape if hasattr(x,'shape') else type(x) for x in out])